AmirAli Amini
610399102

Report:

STEP 1:
    At first in (__init__) function we constructor method that initializes the Pacman object. It takes several parameters:
        1. n and m are the dimensions of the Pacman board.
        2. discount_factor is a value between 0 and 1 that determines the importance of future rewards in the Q-learning algorithm.
        3. learning_rate is a value between 0 and 1 that determines the rate at which the Q-values are updated.
        4. initial_state is the initial position of Pacman on the board.
        5. rewards is a dictionary containing the rewards for each type of cell on the board.


STEP 2:
    Next function (input_board_matrix(self, matrix, initial_state=[0,1])): allows us to input a custom board matrix for the Pacman game. It takes a matrix and sets it as the game board and put a extera lelments around it. then compute the initial_state that parameter allows you to specify the initial position of Pacman on the board and the number of dots .


STEP 3:
    move(self, randomness=0.2, printing=False): This method is responsible for moving Pacman on the board. It takes two optional parameters:
        1.randomness determines the probability of Pacman making a random move instead of following the Q-values.
        2.printing is a boolean that controls whether to print the board after each move.
    The method uses a Q-learning algorithm to update the Q-values based on the rewards and Pacman's movements. It returns the number of steps taken.


STEP 4:
    train(self, n): This method trains Pacman by repeatedly calling the move() method. It takes an integer n that represents the number of training iterations.


STEP 5:
    print_board(self, board, step): This method prints the current state of the board. It takes the board as a parameter and an optional step parameter to indicate the current step.


STEP 6:
    prnt(self): This method prints the current state of the board and the Q-matrix.



finally These functions together create a Pacman game environment and provide methods for training and playing the game using a basic Q-learning algorithm.


In [7]:
import numpy as np
import copy
import random
import networkx as nx
import matplotlib.pyplot as plt

In [8]:
class Pacman:
    def __init__(self, n, m,discount_factor=0.25, learning_rate = 1 ,initial_state = [0,0] , rewards={
            "W" :-10,
            "D" : 1,
            "E" : -0.1,
            "w" :-10,
            "d" : 1,
            "e" : -0.1,
            "A" : 0 ,
            "a" : 0,
            "G" : -10,
            "g" : -10,
            "o" : -10,
            "O" : -10,
        }):
        # acitons:
        #   down:0
        #   right:1
        #   up:2
        #   left:3

        self.n=n+2
        self.m=m+2
        self.board = np.array([["D"]*(m+2 )for _ in range (n+2)])
        self.q_matrix = np.array([[0]*(self.n * self.m) for _ in range (4)],dtype=float)
        print(self.q_matrix.max(axis=0))

        self.initial_state = initial_state
        self.current_state = [0,0]
        
        self.discount_factor=discount_factor
        self.learning_rate=learning_rate

        self.number_of_dots = 0

        self.rewards =rewards
        self.rewards["o"]=-10
        self.rewards["O"]=-10


    def input_board_matrix(self, matrix , initial_state=[0,1]):
        self.initial_state=[initial_state[0]+1,initial_state[1]+1 ]
        for row in matrix:
            row[0:0] = ["O"]
            row.append("O")
        matrix[0:0] = [["O"]*(self.m)]
        matrix.append (["O"]*(self.m))
        for row in matrix:
                print (row)
        self.board = copy.copy(np.array(matrix))
        for row in range(len(matrix)):
            for column in range(len(matrix[row])):
                if matrix[row][column] == "D" or matrix[row][column] == "d" :
                    self.number_of_dots +=1
                elif matrix[row][column] == "A" or matrix[row][column] == "a" :
                    self.initial_state = [row,column]

    def move(self,randomness=0.2,printing=False):
        current_state = self.initial_state
        dots = self.number_of_dots
        board = copy.deepcopy(self.board)
        current_move=0
        step = 0
        while (dots):
            step +=1
            action_reward = -1
            tries = 10
            next_state=[0,0]
            while((action_reward<1 and 5<tries) or (action_reward<0 and tries)):
                tries -=1
                chance = random.random()
                if chance<randomness : 
                    current_move:random.randint(0,3)
                else :
                    m = 0
                    rewards = []
                    next_state = [current_state[0]+1,current_state[1]]
                    rewards.append(self.rewards[board[next_state[0],next_state[1]]])
                    next_state = [current_state[0],current_state[1]+1]
                    rewards.append(self.rewards[board[next_state[0],next_state[1]]])
                    next_state = [current_state[0]-1,current_state[1]]
                    rewards.append(self.rewards[board[next_state[0],next_state[1]]])
                    next_state = [current_state[0],current_state[1]-1]
                    rewards.append(self.rewards[board[next_state[0],next_state[1]]])
                    column = self.q_matrix[:,current_state[0]*self.m + current_state[1]]
                    # print(rewards)
                    for j in range(1,4):
                        if (column[j]+rewards[j]>= column[m]+rewards[m]) : m = j
                    current_move= m
                if (current_move== 0):
                    next_state = [current_state[0]+1,current_state[1]]
                elif (current_move== 1):
                    next_state = [current_state[0],current_state[1]+1]
                elif (current_move== 2):
                    next_state = [current_state[0]-1,current_state[1]]
                elif (current_move== 3):
                    next_state = [current_state[0],current_state[1]-1]

                # print ("c_state: " , current_state , "n_state: " , next_state, "move: ", current_move)
                action_reward = self.rewards[board[next_state[0],next_state[1]]]
                break
            addition_Q_value =   self.learning_rate*(action_reward + self.discount_factor*(self.q_matrix.max(axis=0)[next_state[0]*self.m +next_state[1]]))
            self.q_matrix[current_move][current_state[0]*self.m + current_state[1]] +=addition_Q_value 
            # print("additional value : ", addition_Q_value,self.q_matrix[current_move][current_state[0]*self.m + current_state[1]])
            if board[next_state[0],next_state[1]] =="D" or board[next_state[0],next_state[1]] =="d":
                dots-=1
            elif  board[next_state[0],next_state[1]] in ['o','O','g','G']:
                if printing:self.print_board(board,step)
                return step
            #     break
            if ( board[next_state[0],next_state[1]] not in ['w','W']):
                board[current_state[0]][current_state[1]]="E"
                current_state = copy.copy(next_state)
                board[current_state[0]][current_state[1]]="A"
                if (not dots):print ('dots ended')
            # self.print_board(board,step)
            if printing:self.print_board(board,step)

        # print(board,"\n\n" )
        return step
        
    def train(self,n):
        for i in range(n) :
            print ( f'iteration {i} ,    step : {self.move(randomness=(n-i)/(2*n) , printing=i < 10 or n-i<10)}')


    def print_board (self,board,step):
        print ("step : ",step)
        dic = {
            "W":'✅',
            "D":'⚪️',
            "E":'⚫️',
            "A":'💠',
            "O":'⬛️',
        }
        for row in board :
            print ([dic[i] for i in row] ,' ')
        print('--------------------------------------------------------------------')

    def prnt(self):
        print ('board: ')
        for row in self.board:
            print(row)

        result = []
        dic = ["👇","👉","👆","👈","✅"]
        for i in range (self.n * self.m):
            m= 0
            for j in range(1,4):
                if (self.q_matrix[j][i]>self.q_matrix[m][i]) : m = j
            result.append(dic[m] if self.q_matrix[m][i] else dic[4])
        result = np.array(result)

        print(result.reshape(self.n , -1))


        # print ('q matrix: ')
        # for i in range (40):
        #     print (f'{i+1} : {self.q_matrix[0][i]} , {self.q_matrix[1][i]} , {self.q_matrix[2][i]} , {self.q_matrix[3][i]}')
        # # print (self.q_matrix)




        

In [ ]:
game = Pacman(5,9,learning_rate=0.2)
game.input_board_matrix([
    ["A","W","D","D","D","D","D","D","W",],
    ["D","W","D","D","W","D","W","D","D"],
    ["D","D","D","D","W","D","W","D","D"],
    ["D","D","D","D","W","D","W","D","D"],
    ["D","D","D","D","W","D","W","D","D"],
])

In [ ]:
game.train(100)

In [ ]:
game.train(1000)
game.prnt()


gamma value is equal to 0.25 and alpha value is equal to 0.5

In [ ]:
test = Pacman(3,6,learning_rate=0.5)
test.input_board_matrix([
    ["A","W","D","D","D","D"],
    ["D","W","D","D","W","D"],
    ["D","D","D","D","W","D"],
])

In [ ]:
test.train(3)

In [ ]:
test.prnt()

gamma value is equal to 0.25 and alpha value is equal to 0.01

In [ ]:
test_case_25 = Pacman(7,9,learning_rate=0.01,discount_factor=0.25 )

In [ ]:
test_case_25.input_board_matrix([
    ["A","D","D","D","D","D","D","D","D",],
    ["D","W","W","W","D","W","W","W","D",],
    ["D","W","D","D","D","D","D","W","D",],
    ["D","D","D","W","E","W","D","D","D",],
    ["D","W","D","W","E","W","D","W","D",],
    ["D","W","D","D","W","D","D","W","D",],
    ["D","D","D","D","D","D","D","D","D",],
])

In [ ]:
test_case_25.train(100)
test_case_25.prnt()


gamma value is equal to 0.5 and alpha value is equal to 0.01

In [ ]:
test_gama_5 = Pacman(7,9 , learning_rate=0.01, discount_factor=0.5 )
test_gama_5.input_board_matrix([
    ["A","D","D","D","D","D","D","D","D",],
    ["D","W","W","W","D","W","W","W","D",],
    ["D","W","D","D","D","D","D","W","D",],
    ["D","D","D","W","E","W","D","D","D",],
    ["D","W","D","W","E","W","D","W","D",],
    ["D","W","D","D","W","D","D","W","D",],
    ["D","D","D","D","D","D","D","D","D",],
])

In [ ]:
test_gama_5.train(100)

In [ ]:
test_gama_5.prnt()

gamma value is equal to 1 and alpha value is equal to 0.02

In [ ]:
test_gama_1 = Pacman(7,9 , learning_rate=0.02, discount_factor=1 )
test_gama_1.input_board_matrix([
    ["A","D","D","D","D","D","D","D","D",],
    ["D","W","W","W","D","W","W","W","D",],
    ["D","W","D","D","D","D","D","W","D",],
    ["D","D","D","W","E","W","D","D","D",],
    ["D","W","D","W","E","W","D","W","D",],
    ["D","W","D","D","W","D","D","W","D",],
    ["D","D","D","D","D","D","D","D","D",],
])

In [ ]:
test_gama_1.prnt()

Q-Table

In [ ]:
print(game.q_matrix)

1. Explain what the number of states depends on. Can the number of states be reduced?

The number of states is depends on the size of the game board. in my code the total number of states in the Pacman game environment is (n + 2) * (m + 2).Each state corresponds to a specific cell on the board, including the boundary cells.
now we can reduce the number of states by considering only the inner cells of the board as valid states and omite extera walls that we add to environment at first function(def input_board_matrix(self, matrix , initial_state=[0,1])).
By reducing the number of states, you can potentially simplify the game environment and make the learning process more efficient.

2. Define Action, State, Rewards, and Goal State for the problem and provide your explanation.

Action: An action represents a possible move or decision that Pacman can take in the game. In this case, the actions are limited to moving up, down, left, or right on the game board. Each action can lead Pacman from one state to another.

State: A state represents the current configuration or position of Pacman on the game board. It captures the relevant information needed to make decisions and determine the next action. In this case, a state can be represented by the coordinates (row, column) that indicate the position of Pacman on the board.

Rewards: Rewards are values associated with each state-action pair in the game. They represent the immediate benefit or penalty Pacman receives for taking a particular action in a specific state. Rewards can be positive for desirable actions (e.g., eating a dot) and negative for unfavorable actions (e.g., hitting a wall). The rewards can be defined as a dictionary, where each state-action pair maps to a reward value.

Q4

In [13]:
def graph(Q_table,n,m):
        matrix = [[0]*(n*m) for _ in range (n*m)]
        for i in range (1,n-1):
            for j in range (1,m-1):
                currentState = [i,j]
                matrix[(i)*m+j ][(i+1)*m+j ] = Q_table[0][i*m+j]
                matrix[(i)*m+j ][(i)*m+j+1 ] = Q_table[1][i*m+j]
                matrix[(i)*m+j ][(i-1)*m+j ] = Q_table[2][i*m+j]
                matrix[(i)*m+j ][(i)*m+j-1 ] = Q_table[3][i*m+j]
        matrix = np.array(matrix)

        G = nx.from_numpy_array(matrix)

        # Specify the layout for better visualization (you can try other layouts as well)
        pos = nx.spring_layout(G)

        # Draw the graph with edge labels
        nx.draw(G, pos, with_labels=True, font_weight='bold', node_size=700, node_color='skyblue', font_size=8)
        labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

        # Show the plot
        plt.show()

In [ ]:
test_graph=Pacman(7,9,learning_rate=0.01)
test_graph.input_board_matrix([
    ["A","D","D","D","D","D","D","D","D",],
    ["D","W","W","W","D","W","W","W","D",],
    ["D","W","D","D","D","D","D","W","D",],
    ["D","D","D","W","E","W","D","D","D",],
    ["D","W","D","W","E","W","D","W","D",],
    ["D","W","D","D","W","D","D","W","D",],
    ["D","D","D","D","D","D","D","D","D",],
])
test_graph.train(50)

In [ ]:
Q_table, n , m = test_graph.q_matrix,test_graph.n,test_graph.m
graph(Q_table, n , m )